# Rating optimization with draftsimtools
daniel.brooks@alumni.caltech.edu <br>
January 7, 2018 <br>

In [1]:
import numpy as np
from sklearn import preprocessing

import draftsimtools as ds

In [2]:
#Load 1000 M19 drafts.
m19_set = ds.create_set("data/m19_rating.tsv", "data/m19_land_rating.tsv")
raw_drafts = ds.load_drafts("data/m19_1000drafts.csv")

In [3]:
#Fix commas in card names.
m19_set, raw_drafts = ds.fix_commas(m19_set, raw_drafts)

In [4]:
#Store rating information in a dictionary.
rating_dict = ds.create_rating_dict(m19_set)

In [5]:
#Process the draft data.
drafts = ds.process_drafts(raw_drafts)

Processing draft: 0.


In [6]:
#Create an M19 player. 
#b = ds.SGDBot(rating_dict)

In [7]:
#%%time
#Optimize rating parameters using stochastic gradient descent. (1 minute / 1000 drafts)
#for x in range(10):
#    b.sgd_optimization(drafts[0:25], 0.05)
#print("Done.")

In [8]:
#Write new ratings to file.
#b.write_rating_dict("sgd_05_linear.tsv")
#b.write_error("error.csv")

# PyTorch Preprocessing

In [9]:
#Save draft data as a tensor.
def create_le(cardnames):
    """Create label encoder for cardnames."""
    le = preprocessing.LabelEncoder()
    le.fit(cardnames)
    return le

def draft_to_matrix(cur_draft, le, pack_size=15):
    """Transform draft from cardname list to one hot encoding."""
    pick_list = [np.append(le.transform(cur_draft[i]), (pack_size-len(x))*[0]) \
                 for i, x in enumerate(cur_draft)]
    pick_matrix = np.int16(pick_list)
    return pick_matrix

def drafts_to_tensor(drafts, le, pack_size=15):
    """Create tensor of shape (num_drafts, 45, 15)."""
    pick_tensor_list = [draft_to_matrix(d, le) for d in drafts]
    pick_tensor = np.int16(pick_tensor_list)
    return pick_tensor

#Create drafts tensor.
le = create_le(m19_set["Name"].values)
drafts_tensor = drafts_to_tensor(drafts, le)

In [10]:
#Create torch dataset class.
from torch.utils.data.dataset import Dataset

#Drafts dataset class.
class DraftDataset(Dataset):
    """Defines a draft dataset in PyTorch."""
    
    def __init__(self, drafts_tensor, le):
        """Initialization."""
        self.drafts_tensor = drafts_tensor
        self.le = le
        self.cards_in_set = len(self.le.classes_)
        self.pack_size = int(self.drafts_tensor.shape[1]/3)
        
    def __getitem__(self, index):
        """Return a training example.
        """
        #Compute number of picks in a draft.
        draft_size = self.pack_size*3
        
        #Grab information on current draft.
        pick_num = index % draft_size #0-44
        draft_num = int((index - pick_num)/draft_size)
        
        #Generate 
        x = self.create_x(pick_num, draft_num)
        y = self.create_y(pick_num, draft_num)
        return x, y
    
    def create_x(self, pick_num, draft_num):
        """Generate x, a collection input vector.
        """
        #Initialize collection vector.
        x = np.int16([0] * self.cards_in_set)
        
        #Collect cards picked this draft.
        for n in self.drafts_tensor[draft_num, :(pick_num+1), 0]:
            x[n] += 1
        return x
    
    def create_y(self, pick_num, draft_num, not_in_pack=0.5):
        """Generate y, a target pick vector.
        Picked card is assigned a value of 1.
        Unpicked cards are assigned a value of 0.
        Cards not in the pack are assigned a value of not_inpack.
        """
        #Initialize target vector.
        y = np.array([not_in_pack] * self.cards_in_set)
        
        #Add unpicked cards. 
        cards_in_pack =  self.pack_size - pick_num%self.pack_size
        for n in self.drafts_tensor[draft_num, pick_num, 1:cards_in_pack]: #Pack 1.
            y[n] = 0
            
        #Add picked card.
        y[self.drafts_tensor[draft_num, pick_num, 0]] = 1
        return y
    
    def __len__(self):
        return len(self.drafts)

In [11]:
#Show an example pick representation.
d = DraftDataset(drafts_tensor, le)
x,y = d.__getitem__(15)
print("x =", x, "\n")
print("y =", y)

x = [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 3 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0] 

y = [0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.  0.5 0.5 0.5 0.5 0.5
 0.5 0.5 0.  0.5 0.5 0.5 0.5 0.5 0.5 0.  0.5 0.5 0.5 0.5 0.5 0.  0.5 0.5
 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5
 0.  0.5 0.5 0.  0.  0.5 0.  0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5
 0.5 0.5 0.5 0.5 0.5 0.  0.5 0.  0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5
 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0

In [12]:
#Implement CV environment.

In [13]:
#Explore PyTorch architectures.